# 0.0 Imports

In [1]:
import pandas as pd

In [2]:
import findspark
findspark.init() 

In [3]:
import pyspark
import pyspark.pandas as ps
from pyspark import SparkConf
from pyspark.sql import SparkSession

/home/lara/.pyenv/versions/3.8.8/envs/cds_projects/lib/python3.8/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [6]:
spark = SparkSession \
    .builder \
    .appName("UplifitingModeling") \
    .getOrCreate()

## 0.1 Carregamento dos dados

In [12]:
treatment = pd.read_csv('../data/uplift_train.csv')
clients = pd.read_csv('../data/clients.csv')
products = pd.read_csv('../data/products.csv')
purchases_agg = pd.read_csv('../data/purchases_agg.csv')

In [7]:
purchases = spark.read.option("header",True).csv("../data/purchases.csv").createOrReplaceTempView("Purchases")

# 1.0 Descrição dos dados

## 1.1 Tratamento
Dataset sobre aplicação do tratamento (comunicação via whatsapp) x resultado (compra)

In [6]:
treatment.shape

(200039, 3)

In [7]:
treatment.head()

,client_id,treatment_flg,target
0,000012768d,0,1
1,000036f903,1,1
2,00010925a5,1,1
3,0001f552b0,1,1
4,00020e7b18,1,1


In [8]:
treatment.client_id.nunique()

200039

## 1.2 Clients

In [9]:
clients.shape

(400162, 5)

In [10]:
clients.head()

,client_id,first_issue_date,first_redeem_date,age,gender
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
1,000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
2,000048b7a6,2018-12-15 13:33:11,NaN,68,F
3,000073194a,2017-05-23 12:56:14,2017-11-24 11:18:01,60,F
4,00007c7133,2017-05-22 16:17:08,2018-12-31 17:17:33,67,U


In [11]:
clients.describe()

,age
count,400162.000000
mean,46.488112
std,43.871218
min,-7491.000000
25%,34.000000
50%,45.000000
75%,59.000000
max,1901.000000


## 1.3 Products

In [12]:
products.shape

(43038, 11)

In [13]:
products.head()

,product_id,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
0,0003020d3c,c3d3a8e8c6,c2a3ea8d5e,b7cda0ec0c,6376f2a852,123.0,394a54a7c1,9eaff48661,0.400,0,0
1,0003870676,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,acd3dd483f,10486c3cf0,0.680,0,0
2,0003ceaf69,c3d3a8e8c6,f2333c90fb,419bc5b424,f6148afbc0,271.0,f597581079,764e660dda,0.500,0,0
3,000701e093,ec62ce61e3,4202626fcb,88a515c084,48cf3d488f,172.0,54a90fe769,03c2d70bad,0.112,0,0
4,0007149564,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,63417fe1f3,f329130198,0.600,0,0


In [14]:
products.nunique()

product_id          43038
level_1                 3
level_2                42
level_3               201
level_4               790
segment_id            116
brand_id             4296
vendor_id            3193
netto                 780
is_own_trademark        2
is_alcohol              2
dtype: int64

## 1.4 Purchases

In [15]:
spark.sql("SELECT * FROM Purchases").show(5)

+----------+--------------+--------------------+-----------------------+-----------------------+--------------------+--------------------+------------+----------+----------+----------------+----------------+----------------+
| client_id|transaction_id|transaction_datetime|regular_points_received|express_points_received|regular_points_spent|express_points_spent|purchase_sum|  store_id|product_id|product_quantity|trn_sum_from_iss|trn_sum_from_red|
+----------+--------------+--------------------+-----------------------+-----------------------+--------------------+--------------------+------------+----------+----------+----------------+----------------+----------------+
|000012768d|    7e3e2e3984| 2018-12-01 07:12:45|                   10.0|                    0.0|                 0.0|                 0.0|      1007.0|54a4a11a29|9a80204f78|             2.0|            80.0|            null|
|000012768d|    7e3e2e3984| 2018-12-01 07:12:45|                   10.0|                    0.0|    

In [16]:
spark.sql("SELECT COUNT(*) FROM Purchases").show()

+--------+
|count(1)|
+--------+
|45786568|
+--------+



In [17]:
spark.sql("SELECT COUNT(DISTINCT client_id) AS no_clients, MIN(transaction_datetime) AS primeira_transacao, MAX(transaction_datetime) AS ultima_transacao FROM Purchases").show()

+----------+-------------------+-------------------+
|no_clients| primeira_transacao|   ultima_transacao|
+----------+-------------------+-------------------+
|    400162|2018-11-21 21:02:33|2019-03-18 23:40:03|
+----------+-------------------+-------------------+



In [11]:
purchases_agg.head()

,client_id,last_transaction_datetime,n_transactions,mean_product_quantity,n_stores,n_products,regular_points_received,express_points_received,regular_points_spent,express_points_spent,total_amount_spent
0,3898c89b23,2019-03-18 17:29:35 UTC,50,1.162162,4,118,74.8,0,0,0,12983.55
1,63b5b2f559,2019-03-15 15:47:19 UTC,63,1.540816,3,96,54.9,0,-61,-30,8904.00
2,8db831e9e3,2019-03-16 11:09:33 UTC,142,1.309345,12,479,450.6,0,-416,0,56762.33
3,d2f0b9d5a4,2019-03-14 14:48:52 UTC,48,1.054545,3,133,100.8,0,-104,-30,15224.46
4,ca450b2bce,2019-03-14 15:35:28 UTC,47,1.318043,2,239,164.6,0,-169,0,21357.82


## 1.5 Joining the data

In [15]:
type(clients)

pandas.core.frame.DataFrame

In [13]:
df = pd.merge('treatment', 'clients', on='client_id', how='left')

TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed